# 自动求导
- 假设我们想对函数$y = 2x^Tx$关于列向量$x$进行求导

In [22]:
import torch

In [23]:
x = torch.arange(4.0)

x

tensor([0., 1., 2., 3.])

  - 在计算$y$关于$\vec{x}$的梯度之前，我们需要告诉python，用一个地方来存储梯度

In [24]:
x.requires_grad_(True)
x.grad

- 通过调用反向传播来自动计算y关于x每个分量的梯度

In [ ]:
# 现在来计算y
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

In [26]:
y.backward()
x.grad


tensor([ 0.,  4.,  8., 12.])

In [28]:
x.grad == 4 * x

tensor([ True, False, False, False])

In [33]:
# z = 2 * x
# z 
# x.grad.zero_()
# z.backward()
# x.grad

In [ ]:
# 在默认情况下，PyTorch会累计梯度，我们需要清除之前计算的值
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])